<a href="https://colab.research.google.com/github/ricardoV94/ThinkBayesPymc3/blob/master/ThinkBayes_Chapter_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import theano.tensor as tt

## 1.4 The cookie problem

In [2]:
with pm.Model() as m_1_4:
    obs = pm.Data('data', [True])

    # Prior 
    bowl1 = pm.Bernoulli('bowl1', 1/2)

    # Conditional probabality / Likelihood
    p_vanilla = pm.math.switch(bowl1, 3/4, 1/2)
    vanilla = pm.Bernoulli('vanilla', p_vanilla, observed=obs)

    trace_m_1_4 = pm.sample(2000)

Sequential sampling (2 chains in 1 job)
BinaryGibbsMetropolis: [bowl1]
100%|██████████| 2500/2500 [00:00<00:00, 6127.39it/s]


In [3]:
trace_m_1_4['bowl1'].mean(), 3/5

(0.60325, 0.6)

In [4]:
# Now taking an extra Chocolate and Vanilla cookie
with m_1_4:
    pm.set_data({'data': [True, False, True]})
    _trace_m_1_4 = pm.sample(2000)
_trace_m_1_4['bowl1'].mean()

Sequential sampling (2 chains in 1 job)
BinaryGibbsMetropolis: [bowl1]
100%|██████████| 2500/2500 [00:00<00:00, 5265.59it/s]


0.5305

## 1.6 The M&M problem

In [5]:
with pm.Model() as m_1_6:
    obs1 = pm.Data('data1', [0])
    obs2 = pm.Data('data2', [1])

    # Prior 
    bag94_first = pm.Bernoulli('bag94_first', p=1/2)
    bag94 = [20, 10, 0]
    bag96 = [14, 20, 24]

    # Conditional probabality / Likelihood
    p_bags = pm.math.switch(bag94_first, [bag94, bag96], [bag96, bag94])
    bag1 = pm.Categorical('bag1', p=p_bags[0], observed=obs1)
    bag2 = pm.Categorical('bag2', p=p_bags[1], observed=obs2)

    trace_m_1_6 = pm.sample(200)

Only 200 samples in chain.
Sequential sampling (2 chains in 1 job)
BinaryGibbsMetropolis: [bag94_first]
100%|██████████| 700/700 [00:00<00:00, 4549.99it/s]


In [6]:
trace_m_1_6['bag94_first'].mean(), 20/27

(0.74, 0.7407407407407407)

In [7]:
# Now taking a blue M&M as well
results_m_1_6 = []
for obs in (([0, 2], [1]), ([0, 2], [1, 2])):
    with m_1_6:
        pm.set_data({'data1': obs[0], 'data2': obs[1]})
        _trace_m_1_4 = pm.sample(2000, progressbar=False)
    results_m_1_6.append([*obs, _trace_m_1_4['bag94_first'].mean()])

Sequential sampling (2 chains in 1 job)
BinaryGibbsMetropolis: [bag94_first]
Sequential sampling (2 chains in 1 job)
BinaryGibbsMetropolis: [bag94_first]


In [8]:
for r in results_m_1_6: print(r)

[[0, 2], [1], 0.0]
[[0, 2], [1, 2], 0.0]


## 1.7 The Monty Hall problem

In [9]:
with pm.Model() as m_1_7:
    # Prior 
    door = pm.Categorical('door', np.ones(3)/3)

    # Conditional probabality / Likelihood
    p_doorB = pm.math.switch(tt.eq(door, 0), 1/2, 
                             (pm.math.switch(tt.eq(door, 1), 0, 1)))
    doorB = pm.Bernoulli('door_B', p_doorB, observed=True)

    trace_m_1_7 = pm.sample(200)

Only 200 samples in chain.
Sequential sampling (2 chains in 1 job)
CategoricalGibbsMetropolis: [door]
100%|██████████| 700/700 [00:00<00:00, 5045.43it/s]


In [10]:
(trace_m_1_7['door'] == 0).mean(), (trace_m_1_7['door'] == 2).mean(), 1/3, 2/3

(0.3375, 0.6625, 0.3333333333333333, 0.6666666666666666)